In [ ]:
library(bnlearn) |> suppressPackageStartupMessages()
library(Rgraphviz) |> suppressPackageStartupMessages()
library(tidyverse) |> suppressPackageStartupMessages()
library(gtools) |> suppressPackageStartupMessages()
library(bnstruct) |>suppressPackageStartupMessages()
library(parallel) |> suppressPackageStartupMessages()

In [ ]:
factor_remover <- function(dataset) {
    cols <- colnames(dataset)
    for (i in seq_along(cols)) {
        name <- cols[i]
        dataset[[name]] <- as.character(dataset[[name]])
    }
    return(dataset)
}


dataset <- function(dataset) {
    if (class(dataset) == 'data.frame') {
        return(factor_remover(dataset))
    } else {
        df <- as.data.frame(raw.data(dataset))
        colnames(df) <- dataset@variables
        return(df)
    }
}

In [ ]:
node.values <- function(dataset, parents) {
    # retrieves the unique values of the nodes
    #return a list of vectors, 
    # each vector contains the unique values of the corresponding node
    output <- list()
    for (kk in seq_along(parents)) {
        node <- parents[kk]
        node.vals <- unique(dataset[[node]]) #it was levels()
        output[[kk]] <- node.vals
    }
    return(output)
}

In [ ]:
fastalphaijk <- function(i, pii, dataset, nodes.order, smooth=FALSE) {
    actual_node <- nodes.order[i]
    all.nodes <- c(pii, actual_node)
    subset <- dataset[all.nodes]
    unique.instantiations <- expand.grid(node.values(dataset, all.nodes))
    colnames(unique.instantiations) <- colnames(subset)
    subset.counted <- subset |> group_by(across(everything())) |> summarise(n = n(), .groups="drop")
    checking <-  left_join(unique.instantiations, subset.counted, by=colnames(subset), na_matches="na")  |> arrange(across(-last_col()))
    checking$n[is.na(checking$n)]<- 0
    if (smooth) {
        checking$n <- checking$n + 1   # SOFIA's laplace smoothing
    } else {}
    #cat("\n")
    #print(checking)
    #cat( "\n \n")
    return(checking$n)
}

In [ ]:
k2helper <- function(i, pii, dataset, nodes.order, log=TRUE) {
    # the "f" function in the pdf
    #cat("\n------------------\n")
    phi_i <- expand.grid(node.values(dataset, pii)) 
    #cat("\nphi.i")
    #print( phi_i)
    q_i <- dim(phi_i)[1] # works good
    #cat("\nqi.i", q_i)
    i.th.node <- nodes.order[i]
    #cat("\nithnode", i.th.node)
    v_i <- node.values(dataset, i.th.node)
    #cat("\nvi")
    #print(v_i)
    r_i <- length(v_i[[1]])
    #cat("\nri", r_i)
    individual_alphas <- fastalphaijk(i, pii, dataset, nodes.order) # works good
    #cat("\nindividual alphas::",individual_alphas)
    #aijk <- prod(factorial(individual_alphas)) ##
    grouping_alphas <- matrix(individual_alphas, nrow=r_i) # to get the right Nijs
    nij <- colSums(grouping_alphas)
    #print(nij)
    
    produttoria_alpha <- prod(lfactorial(individual_alphas))
    log_produttoria_alpha <- sum(lfactorial(individual_alphas))
    log_numeratore <- sum(lfactorial(rep(r_i-1,q_i))) # r_i - 1 + 1 = r_i
    numeratore <- factorial(r_i-1)
    log_denominatore <- sum(lfactorial(nij + r_i-1)) # nij + r_i - 1 + 1 = nij + r_i
    denominatore <- factorial(nij+r_i-1)
    log_out <- log_numeratore - log_denominatore + log_produttoria_alpha
    out1 <- prod((numeratore/denominatore))*produttoria_alpha
    out2 <- sum(log_out)
    #cat("logout", log_out)
    ifelse(log, out <- out2, out<- out1)
    #cat("\nproduttoria_alpha",produttoria_alpha )
    #cat("\nproduttoria_alpha",produttoria_alpha )
    #print(frazione)
    return(out)
}

In [ ]:
k2 <- function(dataset, nodes.order, upperbound) {
    start <- Sys.time()  # SOFIA

    nodes <- colnames(dataset)           # node names
    net.dag <- empty.graph(nodes=nodes) # network DAG (Directed Acyclic Graph)
    edges <- list()  # ← new: store edges here  SOFIA
    
    
    for (i in seq_along(nodes.order)) {
        actual_node <- nodes.order[i]
        #cat("\n", i,"\n")
        pii <- c()
        p.old <- k2helper(i, pii,dataset, nodes.order)
        proceed <- TRUE
        #cat("\ni::",i, "\n")
        p.new.container <- c()
        while (proceed && length(pii) <= upperbound) {
            predecessors_idx <- i-1
            predecessors <- nodes.order[0:predecessors_idx]
            #cat("\npredecessors::", predecessors, " pii:: ", pii)
            piiuz <- setdiff(predecessors, pii)
            if (length(piiuz) > 1) {
                daddy.s.probs <- c()
                for (daddy in seq_along(piiuz)) {
                    p.new <- k2helper(i, c(pii, piiuz[daddy]), dataset, nodes.order)
                    daddy.s.probs <- c(daddy.s.probs, c(p.new))
                }
                p.new <- max(daddy.s.probs)
                #cat("\npnew", p.new, "pold", p.old)
                eventuale.genitore <-  piiuz[daddy.s.probs == p.new]
            } else {
                p.new <- k2helper(i, piiuz, dataset, nodes.order)
                eventuale.genitore <- piiuz
            }
            #cat("\npiiuz:: ",piiuz,"\np.old:: ", p.old, "\np.new:: ", p.new)    #edge_matrix <- do.call(rbind, edges)

            #roof <- roof+1
            if (p.new > p.old) {
                p.old <- p.new
                pii <- c(pii, eventuale.genitore)
                p.new.container <- c(p.new.container, p.new)
                net.dag <- set.arc(net.dag, from=eventuale.genitore, to=actual_node)
                #ifelse(roof>length(nodes)^length(nodes), proceed <- FALSE, proceed <- TRUE)
            } else  {
                proceed <- FALSE
            }
        }
        #cat('\nNode', nodes.order[i], '--> Parents:', pii,'\n') 
        
        # Store edges: each pii is a parent of actual_node  SOFIA
        if (length(pii) >= 0) {
            for (iterator in seq_along(pii)) {
                parent <- pii[iterator]
                edges[[length(edges) + 1]] <- c(parent, actual_node)
            }
        }

    }
    
    net.score <- score(net.dag, dataset |> mutate_all(as_factor) |> as.data.frame()) # network scor
    # Convert to matrix or return edge list
    edge_matrix <- do.call(rbind, edges)
    
    end <- Sys.time()
    execution_time <- difftime(time1 = end, time2 = start, units = "secs") 
    cat("\nExecution Time:", execution_time, "s")
    
    return(list('edges' = edge_matrix, 
                'dag' = net.dag,
                'score' = net.score))
}

## Explorer Function

IDEA: mandare run con diversi `n_permutations` e vedere come performa K2 + come cambia la rete (vedi 3.B del notebook 2 dei tizi)

In [ ]:
explo_scorer_single <- function(dataset, n_permutations = 1000, upperbound=2) {
  start <- Sys.time()
  
  set.seed(8)
  cols <- colnames(dataset)
  
  ncols <- length(cols)
  
  if(ncols <= 7){
    permutations <- permutations(n = ncols, r = ncols, v = cols)
    row_limit <- nrow(permutations)
  } else {
    # Sample random permutations (shuffles) for large ncols
    row_limit <- n_permutations
  }

  result <- k2(dataset, cols, upperbound = upperbound)
  best.dag <- list(result$dag)
  best.score <- c(result$score)
  best.order <- list(cols)
  
  for(i in 2:row_limit){
    if(ncols <= 7){
      row_permutation <- permutations[i, ]
    } else {
      row_permutation <- sample(cols, size = ncols, replace = FALSE)
    }
    result <- k2(dataset, row_permutation, upperbound=upperbound)
    #if (result$score > best.score) {
    #  best.dag <- result$dag
    #  best.score <- result$score
    #  best.order <- row_permutation
    #}
    best.dag[i] <- result$dag
    best.score[i] <- result$score
    best.order[i] <- row_permutation
    #cat('Running iteration', i, "\n")
  }
  max.score <- max(best.score)   # SOFIA
  idx <- which.max(best.score)
  
  end <- Sys.time()
  execution.time <- difftime(time1 = end, time2 = start, units = "secs")
  #cat("\nExecution Time:", execution.time, "s\n")
  
  # Return best.score or best ordering if needed
  return(list('dag'=best.dag[[idx]], 'order'=best.order[[idx]], 'exec_time'=execution.time, "score" = max.score))  # SOFIA
}

explo_scorer <- function(dataset, n_permutations = 1000, upperbound=2) {
  start <- Sys.time()
  set.seed(8)
  cols <- colnames(dataset)
  ncols <- length(cols)
  
  if(ncols <= 7){
    permutations <- permutations(n = ncols, r = ncols, v = cols)
    row_limit <- nrow(permutations)
    perm_list <- split(permutations, seq(nrow(permutations)))
  } else {
    row_limit <- n_permutations
    perm_list <- lapply(1:row_limit, function(i) sample(cols, size = ncols, replace = FALSE))
  }

  # Parallel evaluation
  results <- mclapply(perm_list, function(row_permutation) {
    result <- k2(dataset, row_permutation, upperbound=upperbound)
    list(dag = result$dag, score = result$score, order = row_permutation)
  }, mc.cores = detectCores() - 1)

  # Extract scores and find the best
  scores <- sapply(results, function(x) x$score)
  max.score <- max(scores)    #SOFIA
  idx <- which.max(scores)
  
  end <- Sys.time()
  execution.time <- difftime(time1 = end, time2 = start, units = "secs")
  
  return(list('dag' = results[[idx]]$dag, 'order' = results[[idx]]$order, 'exec_time' = execution.time, 'score' = max.score))  # SOFIA
}

Gigi prova questa simpatica funzione che io non posso :) <br> va applicata a tutti i dataset

In [ ]:
best.net <- function(data, n_permutations = 1000, upperbound=2) {
    result <- explo_scorer(data, n_permutations, upperbound)
    
    result.dag <- result$dag
    result.order <- result$order
    result.exec.566time <- result$exec_time
    result.score <- result$score
    
    #pdf("learned_network_plot.pdf")
    graphviz.plot(x = result.dag,
                  highlight = NULL,
                  layout = "dot",
                  shape = "circle",
                  fontsize = 12,
                  main = paste("Bayesian Network Learned with K2\nScore:", round(result.score, 3)),
                  render = TRUE)
    #dev.off()
    
    return(result)
}

## Learning Dataset

In [ ]:
learning.result <- best.net(data = dataset(learning.test), n_permutations = 2000, upperbound = 2)

## Asia Dataset

In [ ]:
asia.result <- best.net(data = dataset(asia()), n_permutations = 2000, upperbound = 2)

## Child Dataset

In [ ]:
child.result <- best.net(data = dataset(child()), n_permutations = 2000, upperbound = 2)

## Graphical Comparison

In [ ]:
learning.library.dag <- model2network("[A][C][F][B|A][D|A:C][E|B:F]")
asia.library.dag <- model2network("[Asia][Smoke][Tubercolosys|Asia][LungCancer|Smoke][Bronchitis|Smoke][Dyspnea|Bronchitis:Either][Either|Tubercolosys:LungCancer][X-ray|Either]")
child.model.string <- paste0(
  "[BirthAsphyxia][Disease|BirthAsphyxia][Sick|Disease][Age|Disease:Sick][DuctFlow|Disease]",
    "[CardiacMixing|Disease][LungParench|Disease][LungFlow|Disease][LVH|Disease]",
    "[Grunting|Sick:LungParench][HypDistrib|DuctFlow:CardiacMixing][HypoxiaInO2|CardiacMixing:LungParench][CO2|LungParench][ChestXray|LungParench:LungFlow][LVHReport|LVH]",
    "[GruntingReport|Grunting][LowerBodyO2|HypDistrib:HypoxiaInO2][RUQO2|HypoxiaInO2][CO2Report|CO2][XrayReport|ChestXray]")

child.library.dag <- model2network(child.model.string)

In [ ]:
graphviz.compare(learning.library.dag, learning.result$dag, shape = "circle")

In [ ]:
graphviz.compare(asia.library.dag, asia.result$dag, shape = "circle")

In [ ]:
graphviz.compare(child.library.dag, child.result$dag, shape = "circle")

# Part 2: bnlearn library
Implement and test the K2 algorithm with the test data sets ([3]). Compare the results with that obtained with the bnstruct R library [4]

## Useful function to implement to avoid repetition

Here we need **BNDataset** as input for the `learn.network` function. This function comes from `bnstruct` package, which works with **BN** and **BNDataset** objects, meanwhile `bnlearn` works with **bn** objects.<br>So here it is a function to convert **data frames** $\rightarrow$ **BNDataset**

In [ ]:
# from data.frame to BNDataset
df.to.BND <- function(dataframe) {
    df_name <- deparse(substitute(dataframe))  # to retrieve the name as a string
    new.BNDataset <- BNDataset(data = dataframe,
                          discreteness = rep("d", ncol(dataframe)),
                          variables = colnames(dataframe),
                          node.sizes = c(sapply(dataframe, function(col) length(unique(col)))),
                          starts.from = 0,  # otherwise it starts from 1 to |x| that here is also 1
                          bndataset.name = paste0(df_name, ".dataset"))
    return(new.BNDataset)
}                   

**PROBLEM**: `learn.network` returns an object of class **BN** which is not compatible with `graphviz.plot()` from `bnlearn` (which - again - takes in input **bn** objects), so i used `plot` instead.<br>The alternative solution is to convert from **BN** $\rightarrow$ **bn** manually extracting the adjacency matrix or arc list and rebuild the structure with `bnlearn` $\Rightarrow$ too much *sbatti*, i don't like it and for me doesn't deserve the *pena*.

In [ ]:
learn_network <- function(dataset, impute = FALSE, bootstrap = FALSE, algo="mmhc", score.fun="BDeu") {     #takes in input BNDataset
    start <- Sys.time()
    
    if (impute) {
        message("Imputing dataset")
        dataset <- impute(dataset)
    }

    raw <- raw.data(dataset)
      
    if (impute && bootstrap) {
        message("Learning network imputing NA, then bootstrap.")
        net <- learn.network(dataset, 
                             use.imputed.data = TRUE,
                             bootstrap = TRUE, 
                             algo = algo,
                             score.fun = score.fun)
    } else if (impute) {
        message("Learning network using imputed data.")
        net <- learn.network(dataset, 
                             use.imputed.data = TRUE,
                             algo = algo,
                             score.fun = score.fun)
    } else if (bootstrap) {
        message("Learning network using bootstrap.")
        net <- learn.network(dataset, 
                             bootstrap = TRUE,
                             algo = algo,
                             score.fun = score.fun)
    } else {
        message("Learning network using default method.")
        net <- learn.network(dataset,
                             algo = algo,
                             score.fun = score.fun)
    }
    
    # grafichino
    # if we want to be sboroni we can save the fig and add it to the presentation ;)
    #pdf("learned_network_plot.pdf")
    plot(net)
    #dev.off()
    end <- Sys.time()
    execution_time <- difftime(time1 = end, time2 = start, units = "secs") 
    cat("\nExecution Time:", execution_time, "s")
        
    return(net)
}

## Ruiz Dataset

In [ ]:
ruiz.dataset <- data.frame(x1 = c(1, 1, 0, 1, 0, 0, 1, 0, 1, 0),
                          x2 = c(0, 1, 0, 1, 0, 1, 1, 0, 1, 0),
                          x3 = c(0, 1, 1, 1, 0, 1, 1, 0, 1, 0)) |> df.to.BND()

In [ ]:
ruiz.net <- learn_network(dataset = ruiz.dataset)

## learning.test Dataset

In [ ]:
learning.dataset <- data.frame(lapply(learning.test, function(col) as.integer(as.factor(col)) - 1)) |> df.to.BND()

In [ ]:
learning.net <- learn_network(dataset = learning.dataset)

## Asia Dataset

In [ ]:
asia_net <- learn_network(dataset = asia(), impute = FALSE, bootstrap = FALSE)

In [ ]:
asia_net <- learn_network(dataset = asia_data, impute = FALSE, bootstrap = TRUE)   # default value is num.boots = 100

## Child Dataset

In [ ]:
child_data <- child()
child_net <- learn_network(dataset = child_data, impute = FALSE, bootstrap = FALSE)

In [ ]:
child_net <- learn_network(dataset = child_data, impute = TRUE, bootstrap = FALSE)